In [3]:
import torch
import torch.nn as nn
from tqdm import tqdm
import os
import numpy as np
from spikingjelly.clock_driven import functional 

# Custom imports (Must exist in your directory)
from snn_model import SimpleSNN
from data_loader import create_dataloaders

# ===========================
# Utils: SI-SDR & Loss
# ===========================
def compute_si_sdr(estimate, reference, epsilon=1e-8):
    estimate = estimate - estimate.mean()
    reference = reference - reference.mean()
    
    reference_energy = torch.sum(reference ** 2) + epsilon
    optimal_scaling = torch.sum(estimate * reference) / reference_energy
    
    projection = optimal_scaling * reference
    noise = estimate - projection
    
    ratio = torch.sum(projection ** 2) / (torch.sum(noise ** 2) + epsilon)
    return 10 * torch.log10(ratio)

def batch_pit_mse_loss(pred1, pred2, true1, true2):
    b_size = pred1.shape[0]
    p1 = pred1.reshape(b_size, -1)
    p2 = pred2.reshape(b_size, -1)
    t1 = true1.reshape(b_size, -1)
    t2 = true2.reshape(b_size, -1)
    
    loss_a = torch.mean((p1 - t1) ** 2, dim=1) + torch.mean((p2 - t2) ** 2, dim=1)
    loss_b = torch.mean((p1 - t2) ** 2, dim=1) + torch.mean((p2 - t1) ** 2, dim=1)
    
    best_loss, _ = torch.min(torch.stack([loss_a, loss_b], dim=1), dim=1)
    return best_loss.mean()

# ===========================
# Config & Setup
# ===========================
DATA_DIR = r'./LibriMix/output'
BATCH_SIZE = 32
HIDDEN_SIZE = 512
T_STEPS = 6
LR = 1e-3
EPOCHS = 50
N_FFT = 512
HOP_LENGTH = 160
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Device: {DEVICE}")
print(f"Model: Simple SNN (T={T_STEPS}) + SI-SDR Eval")

# Load Data
train_loader, test_loader, _, _ = create_dataloaders(DATA_DIR, BATCH_SIZE, 0.15, 0)

# Initialize Model
model = SimpleSNN(input_size=257, hidden_size=HIDDEN_SIZE, T=T_STEPS).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

os.makedirs('save_models', exist_ok=True)
best_sdr = -float('inf')

# ===========================
# Training Loop
# ===========================
for epoch in range(EPOCHS):
    # --- Training ---
    model.train()
    epoch_loss = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]")
    
    for mix, s1, s2 in pbar:
        mix, s1, s2 = mix.to(DEVICE), s1.to(DEVICE), s2.to(DEVICE)
        
        # 1. Ground Truth Mask Calculation
        with torch.no_grad():
            win = torch.hann_window(N_FFT).to(DEVICE)
            mix_stft = torch.stft(mix, N_FFT, HOP_LENGTH, return_complex=True, window=win)
            mix_mag = torch.abs(mix_stft)
            mix_input = torch.log1p(mix_mag) 
            
            s1_mag = torch.abs(torch.stft(s1, N_FFT, HOP_LENGTH, return_complex=True, window=win))
            s2_mag = torch.abs(torch.stft(s2, N_FFT, HOP_LENGTH, return_complex=True, window=win))
            
            denom = s1_mag + s2_mag + 1e-8
            m1_true = s1_mag / denom
            m2_true = s2_mag / denom

        # 2. Forward
        functional.reset_net(model)
        m1_pred, m2_pred = model(mix_input)
        
        # 3. Loss & Backward
        loss = batch_pit_mse_loss(m1_pred, m2_pred, m1_true, m2_true)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        
        epoch_loss += loss.item()
        pbar.set_postfix({'MSE': f'{loss.item():.4f}'})
        
    avg_loss = epoch_loss / len(train_loader)

    # --- Evaluation (SI-SDR) ---
    model.eval()
    sdr_sum = 0
    eval_pbar = tqdm(test_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Eval]")
    
    with torch.no_grad():
        for mix, s1, s2 in eval_pbar:
            mix, s1, s2 = mix.to(DEVICE), s1.to(DEVICE), s2.to(DEVICE)
            
            # STFT
            win = torch.hann_window(N_FFT).to(DEVICE)
            mix_stft = torch.stft(mix, N_FFT, HOP_LENGTH, return_complex=True, window=win)
            mix_mag = torch.abs(mix_stft)
            mix_phase = torch.angle(mix_stft)
            mix_input = torch.log1p(mix_mag)

            # Predict
            functional.reset_net(model)
            m1_pred, m2_pred = model(mix_input)
            
            # Reconstruct
            est_mag1 = mix_mag * m1_pred
            est_mag2 = mix_mag * m2_pred
            
            est_s1 = torch.istft(est_mag1 * torch.exp(1j * mix_phase), N_FFT, HOP_LENGTH, window=win)
            est_s2 = torch.istft(est_mag2 * torch.exp(1j * mix_phase), N_FFT, HOP_LENGTH, window=win)
            
            # Align lengths
            min_len = min(mix.shape[-1], est_s1.shape[-1])
            est_s1, est_s2 = est_s1[..., :min_len], est_s2[..., :min_len]
            s1, s2 = s1[..., :min_len], s2[..., :min_len]

            # Calculate SI-SDR (PIT)
            for i in range(mix.shape[0]):
                val_sdr1 = (compute_si_sdr(est_s1[i], s1[i]) + compute_si_sdr(est_s2[i], s2[i])) / 2
                val_sdr2 = (compute_si_sdr(est_s1[i], s2[i]) + compute_si_sdr(est_s2[i], s1[i])) / 2
                sdr_sum += max(val_sdr1, val_sdr2).item()
    
    avg_sdr = sdr_sum / len(test_loader.dataset)
    
    print(f"Summary -> Train MSE: {avg_loss:.4f} | Test SI-SDR: {avg_sdr:.2f} dB")
    
    if avg_sdr > best_sdr:
        best_sdr = avg_sdr
        torch.save(model.state_dict(), 'save_models/best_snn.pt')
        print(f"Saved Best SNN (SDR: {best_sdr:.2f} dB)")

Device: cuda
Model: Simple SNN (T=6) + SI-SDR Eval
Found 15000 valid files


Epoch 1/50 [Eval]: 100%|███████████████████████████████████████████████████████████████| 71/71 [00:05<00:00, 12.48it/s]


Summary -> Train MSE: 0.2994 | Test SI-SDR: 0.23 dB
Saved Best SNN (SDR: 0.23 dB)


Epoch 2/50 [Eval]: 100%|███████████████████████████████████████████████████████████████| 71/71 [00:05<00:00, 13.08it/s]


Summary -> Train MSE: 0.2407 | Test SI-SDR: 0.66 dB
Saved Best SNN (SDR: 0.66 dB)


...

Summary -> Train MSE: 0.2162 | Test SI-SDR: 2.34 dB
Saved Best SNN (SDR: 2.34 dB)


Epoch 49/50 [Eval]: 100%|██████████████████████████████████████████████████████████████| 71/71 [00:05<00:00, 14.13it/s]


Summary -> Train MSE: 0.2159 | Test SI-SDR: 2.34 dB


Epoch 50/50 [Eval]: 100%|██████████████████████████████████████████████████████████████| 71/71 [00:05<00:00, 12.52it/s]


Summary -> Train MSE: 0.2158 | Test SI-SDR: 2.28 dB
